# Testing CHIANTI data in the Cloud

In [1]:
import h5pyd
import h5py
import numpy as np
import astropy.units as u

/Users/willbarnes/anaconda/envs/chianti_cloud/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DOMAIN = 'chianti.hdfgroup.org'
ENDPOINT = 'http://127.0.0.1:5000'

In [ ]:
class DataIndexer(object):
    """
    Data access layer for each distinct CHIANTI dataset

    Acts as an interface layer between `Ion` and the CHIANTI data stored in the
    HDF5 database. All data that the user interacts with passes through this layer.

    .. warning:: This object is not meant to be instantiated directly by the user. Rather, instances
                 are created by higher-level objects in order to provide access to the CHIANTI data.

    Parameters
    ----------
    hdf5_path : `str`
    top_level_path : `str`
    """
    
    def __init__(self, hdf5_path, top_level_path):
        self.top_level_path = top_level_path
        self.hdf5_dbase_root = hdf5_path

    @classmethod
    def create_indexer(cls, hdf5_path, top_level_path):
        """
        Create an instance as long as the dataset exists
        """
        with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
            path_is_valid = True if top_level_path in hf else False
        return cls(hdf5_path, top_level_path) if path_is_valid else None

    @property
    def version(self):
        with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
            if 'chianti_version' in hf[self.top_level_path].attrs:
                version = hf[self.top_level_path].attrs['chianti_version']
            else:
                version = None
        return version
    
    @property
    def fields(self):
        with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
            fields = [k for k in hf[self.top_level_path]]
        return fields

    def as_table(self):
        qt = QTable()
        for field in self.fields:
            qt[field] = self[field]
        return qt
    
    def __contains__(self, key):
        with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
            key_in_grp = key in hf[self.top_level_path]
        return key_in_grp
    
    def __getitem__(self, key):
        if type(key) is int:
            raise NotImplementedError('Iteration not supported.')
        with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
            if key not in self:
                return None
            ds = hf[self.top_level_path][key]
            if isinstance(ds, h5py.Group):
                data = DataIndexer.create_indexer(self.hdf5_dbase_root,
                                                  '/'.join([self.top_level_path, key]))
            else:
                if ds.attrs['unit'] == 'SKIP' or ds.dtype == 'object':
                    data = np.array(ds, dtype=ds.dtype)
                else:
                    data = u.Quantity(np.array(ds), ds.attrs['unit'], dtype=ds.dtype)
                if '|S' in data.dtype.str:
                    data = data.astype(str)
        return data
    
    def __repr__(self):

        def ufilter(x):
            return ('' if 'unit' not in x.attrs or x.attrs['unit'] == 'SKIP'
                    or x.attrs['unit'] == '' else '({})'.format(x.attrs['unit']))

        def dfilter(x):
            return '' if 'description' not in x.attrs else '{}'.format(x.attrs['description'])

        with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
            grp = hf[self.top_level_path]
            var_names = [k for k in grp.keys()]
            # var_names = [(key, ufilter(grp[key]), dfilter(grp[key])) for key in grp_keys]
            footer = '' if 'footer' not in grp.attrs else grp.attrs['footer']
            
        #name_strs = '\n'.join(['{} {} -- {}'.format(*v) for v in var_names])
        name_strs = '\n'.join(var_names)
        version = '' if self.version is None else f'-- v{self.version}'
        return f"""{self.top_level_path} {version}

Fields
------
{name_strs}

Footer
------
{footer}"""

In [ ]:
di = DataIndexer(None, '/')

In [ ]:
di['fe/fe_11/elvlc']

In [ ]:
di['fe/fe_11/elvlc/multiplicity']

In [ ]:
di.version

In [ ]:
di.fields

In [ ]:
di['al']

In [ ]:
with h5py.File(DOMAIN, 'r', endpoint=ENDPOINT) as hf:
    grp = hf['/']
    print([a for a in grp.attrs])
    #print([k for k in hf['al'].keys()])

In [ ]:
var_names

In [9]:
with h5py.File('/Users/willbarnes/Documents/projects/h5serv/data/test/tall.h5','r') as hf:
    print([k for k in hf['g1/g1.1'].keys()])
    foo = np.array(hf['g1/g1.1/dset1.1.1'])

['dset1.1.1', 'dset1.1.2']


In [10]:
foo

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18],
       [ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27],
       [ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36],
       [ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45],
       [ 0,  6, 12, 18, 24, 30, 36, 42, 48, 54],
       [ 0,  7, 14, 21, 28, 35, 42, 49, 56, 63],
       [ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72],
       [ 0,  9, 18, 27, 36, 45, 54, 63, 72, 81]], dtype=int32)

In [45]:
with h5py.File('/Users/willbarnes/Documents/projects/h5serv/data/chianti.h5','r') as hf:
    # print([k for k in hf['g1/g1.1'].keys()])
    foo = np.array(hf['h/h_1/elvlc/E_obs'])

In [46]:
foo

array([     0.   ,  82258.956,  82258.921,  82259.287,  97492.224,
        97492.213,  97492.321,  97492.321,  97492.357, 102823.855,
       102823.851, 102823.896, 102823.896, 102823.911, 102823.911,
       102823.919, 105291.633, 105291.631, 105291.654, 105291.654,
       105291.662, 105291.662, 105291.666, 105291.666, 105291.668])

In [47]:
with h5pyd.File('tall.test.hdfgroup.org','r',endpoint='http://127.0.0.1:5000') as hf:
    print([k for k in hf['g1/g1.1'].keys()])
    foo = np.array(hf['g1/g1.1/dset1.1.1'])

['dset1.1.1', 'dset1.1.2']


In [48]:
foo

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18],
       [ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27],
       [ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36],
       [ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45],
       [ 0,  6, 12, 18, 24, 30, 36, 42, 48, 54],
       [ 0,  7, 14, 21, 28, 35, 42, 49, 56, 63],
       [ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72],
       [ 0,  9, 18, 27, 36, 45, 54, 63, 72, 81]], dtype=int32)

In [53]:
with h5pyd.File('chianti.hdfgroup.org','r',endpoint='http://127.0.0.1:5000') as hf:
    #print([k for k in hf['g1/g1.1'].keys()])
    foo = np.array(hf['fe/fe_15/ioneq/chianti/ionization_fraction'])

In [54]:
foo

array([2.31584178e+077, 2.31584178e+077, 9.33784071e-322,             nan,
       2.59903817e-144, 1.65780439e-076, 9.72157424e-072, 6.20013304e-091,
       6.86422360e+169, 2.59027864e-144, 1.20649766e+285, 1.16442050e-028,
       2.80867394e+059, 5.21893390e+164, 3.08395988e+039, 8.70336807e+150,
       2.18232560e-094, 5.49409691e-143, 6.09114655e+247, 6.93627722e+092,
       1.36449813e-143, 9.95718346e-143, 7.45764518e+295, 3.98568237e+252,
       4.26241819e+180, 1.94920553e-153, 3.32107505e+098, 3.08395988e+039,
       1.76192294e+016, 3.71879460e+006, 5.29435892e+199, 1.16096349e-028,
       9.97830543e+044, 8.71617999e+150, 1.04916846e-153, 1.04857810e-142,
       6.09114655e+247, 3.92579378e+020, 6.51946094e+102, 1.06244775e-153,
       4.24510207e+175, 5.05117709e-038, 6.98340586e-076, 3.75560927e+126,
       8.88060621e+252, 4.23661523e+175, 5.05117709e-038, 6.98340586e-076,
       3.75560927e+126, 2.18226585e+243, 4.69419004e+257, 1.05146958e-153,
       1.16440800e-028, 7